In [7]:
import env
import pandas as pd
import numpy as np
import utilities as utils

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import requests
import nltk
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
import unicodedata
import re
import pandas as pd
import numpy as np

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


from wordcloud import WordCloud
from PIL import Image
from wordcloud import ImageColorGenerator
from jupyterthemes import jtplot
jtplot.style(grid=False)

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

from wrangle import wrangle_articles

In [3]:
all_articles_df, only_articles_with_text_df = wrangle_articles()

/Users/ken/codeup-data-science/fake_news_capstone/wrangle.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles_df.date[row] = "Aug 16, 2016"


NaN values
Empty DataFrame
Columns: [rows, columns]
Index: []
--------------------------------
Empty values
      rows  columns
0      120        1
1      122        1
2      152        1
3      165        1
4      172        1
..     ...      ...
627  24267        1
628  25130        1
629  25152        1
630  25701        1
631  25746        1

[632 rows x 2 columns]
--------------------------------


In [4]:
all_articles_df.to_csv("all_articles.csv")
only_articles_with_text_df.to_csv("only_articles_with_text.csv")

In [11]:
all_articles_df.head()

,title,text,subject,date,is_fake
37302,flashback king obama commute sentence 22 drug ...,making room hillary president obama today anno...,politics,2015-03-31,True
37303,apple ceo say religious freedom law dangerous ...,gay mafia new corporate article need read shee...,politics,2015-03-31,True
37304,watch dirty harry reid lie romneys tax didnt win,case missed sen harry reid rnv announced last ...,politics,2015-03-31,True
37305,oh guess funded shrine ted kennedy,nothing like political cronyism make stomach c...,politics,2015-03-31,True
37306,benghazi panel call hillary testify oath white...,anyone really think hillary clinton come clean...,politics,2015-03-31,True


In [14]:
all_articles_df.shape, only_articles_with_text_df.shape

((44888, 5), (44256, 5))

In [15]:
only_articles_with_text_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44256 entries, 37302 to 0
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   title    44256 non-null  object        
 1   text     44256 non-null  object        
 2   subject  44256 non-null  object        
 3   date     44256 non-null  datetime64[ns]
 4   is_fake  44256 non-null  bool          
dtypes: bool(1), datetime64[ns](1), object(3)
memory usage: 3.0+ MB


In [17]:
only_articles_with_text_df.subject.value_counts(normalize=True)

politicsNews       0.253028
worldnews          0.220377
News               0.200741
politics           0.154465
left-news          0.100687
Government News    0.035430
US_News            0.017693
Middle-east        0.017580
Name: subject, dtype: float64

In [18]:
only_articles_with_text_df.is_fake.value_counts()

True     23305
False    20951
Name: is_fake, dtype: int64

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
doc = nlp(text)
doc._.polarity      # Polarity: -0.125
doc._.subjectivity  # Sujectivity: 0.9
doc._.assessments   # Assessments: [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]

In [8]:
nlp = spacy.load('en_core_web_md')
nlp.add_pipe("spacytextblob")
print(nlp.pipe_names) 

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer', 'spacytextblob']


In [21]:
def polarity(text):
    '''function takes a body of text and outputs the polarity score [-1.0:1.0]'''
    doc = nlp(text)
    return doc._.polarity

def subjectivity(text):
    '''fuction takes a body of text and outputs the subjectivity score [0.0:1.0]'''
    doc = nlp(text)
    return doc._.subjectivity    

In [22]:
only_articles_with_text_df['text_polarity'] = only_articles_with_text_df.text.apply(polarity)
only_articles_with_text_df.head()

,title,text,subject,date,is_fake,text_polarity
37302,flashback king obama commute sentence 22 drug ...,making room hillary president obama today anno...,politics,2015-03-31,True,-0.201587
37303,apple ceo say religious freedom law dangerous ...,gay mafia new corporate article need read shee...,politics,2015-03-31,True,-0.040032
37304,watch dirty harry reid lie romneys tax didnt win,case missed sen harry reid rnv announced last ...,politics,2015-03-31,True,0.150000
37305,oh guess funded shrine ted kennedy,nothing like political cronyism make stomach c...,politics,2015-03-31,True,0.122865
37306,benghazi panel call hillary testify oath white...,anyone really think hillary clinton come clean...,politics,2015-03-31,True,0.110586


In [23]:
only_articles_with_text_df['text_subjectivity'] = only_articles_with_text_df.text.apply(subjectivity)
only_articles_with_text_df.head()

,title,text,subject,date,is_fake,text_polarity,text_subjectivity
37302,flashback king obama commute sentence 22 drug ...,making room hillary president obama today anno...,politics,2015-03-31,True,-0.201587,0.493452
37303,apple ceo say religious freedom law dangerous ...,gay mafia new corporate article need read shee...,politics,2015-03-31,True,-0.040032,0.582057
37304,watch dirty harry reid lie romneys tax didnt win,case missed sen harry reid rnv announced last ...,politics,2015-03-31,True,0.150000,0.511111
37305,oh guess funded shrine ted kennedy,nothing like political cronyism make stomach c...,politics,2015-03-31,True,0.122865,0.441116
37306,benghazi panel call hillary testify oath white...,anyone really think hillary clinton come clean...,politics,2015-03-31,True,0.110586,0.433784


In [24]:
only_articles_with_text_df.to_csv("only_articles_with_text.csv")